In [1]:
import os
import glob
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
from pysis import isis

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from knoten.bundle import *

/home/jmapel/.conda/envs/knoten/lib/python3.7/site-packages/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


## Load in Network

In [2]:
!ls /work/projects/control_network_metrics/registration_quality/*.net

/work/projects/control_network_metrics/registration_quality/hand.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_1pts_0mean_0.25std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_1pts_1.5mean_1std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_1pts_5mean_2std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_2pts_0mean_0.25std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_2pts_1.5mean_1std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_2pts_5mean_2std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_4pts_0mean_0.25std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_4pts_1.5mean_1std.net
/work/projects/control_network_metrics/registration_quality/measure_error_25px_4pts_5mean_2std.net
/work/projects/control_ne

In [3]:
cubes = '/work/projects/control_network_metrics/registration_quality/cubes.lis'
sensors = generate_sensors(cubes, directory='data/', clean=True)

In [4]:
base_net_file = '/work/projects/control_network_metrics/registration_quality/hand.net'
base_net = io_controlnetwork.from_isis(base_net_file)

network_dir = '/work/projects/control_network_metrics/registration_quality/'

## Determine Solve Parameters

In [5]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity

## Functions

In [6]:
def compute_sigma(V, W_parameters, W_observations):
    """
    Computes the resulting standard deviation of the residuals for the current state of the bundle network.
    
    Parameters
    ----------
    V  :  np.array
          The control network dataframe with updated ground points
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: point weights)
    
    Returns
    -------
       : float64
         Standard deviation of the residuals
    
    """
    num_parameters = W_parameters.shape[0]
    num_observations = W_observations.shape[0]
    dof = num_observations - num_parameters
    VTPV = (V.dot(W_observations).dot(V))
    sigma0 = np.sqrt(VTPV/dof)
    return sigma0

def bundle_iteration(J, V, W_parameters, W_observations):
    """
    Parameters
    ----------
    J  :  ndarray
          The control network as a dataframe generated by plio.
    V  :  np.array
          The control network dataframe with updated ground points
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: measure weights)
    
    Returns
    -------
    N  :  
    """
    
    N = J.T.dot(W_observations).dot(J) + W_parameters
    C = J.T.dot(W_observations).dot(V)
    dX = np.linalg.inv(N).dot(C)
    return N, dX

# For data snooping we need to calculate updated residuals
def compute_normalized_residual(J, V, N, W_parameters, W_observations):
    """
    Computes the normalized residual statistic for the data snooping method. Method derived from 
    Forstner 1985 "The Reliability of Block Triangulation"
    
    Parameters
    ----------
    V  :  np.array
          The control network dataframe with updated ground points
    N  :  
        
    W_parameters  :  ndarray 
                     The parameter weight matrix (i.e.: sensor parameters and point weights)
    W_observations  :  ndarray
                     The observation weight matrix (i.e.: point weights)
    
    Returns
    -------
       : np.array
         Normalized residual statistic for the data snooping
    
    """
    sigma0 = compute_sigma(V, W_parameters, W_observations)
    Qxx = np.linalg.inv(N)
    Qvv = np.linalg.inv(W_observations) - J.dot(Qxx).dot(J.T)
    qvv = np.diagonal(Qvv)
    sigma_vi = sigma0*np.sqrt(qvv)
    wi = -V/sigma_vi
    
    return wi

In [7]:
def check_network(network):
    """
    Check that all control points in a network have at least 2 remaining measures.
    
    Parameters
    ----------
    network : DataFrame
              The control network as a dataframe generated by plio
    
    Returns
    -------
     : list
       List of measure indices that were masked out for being the only measure on a point.
    """
    masked = []
    for point_id, group in network.groupby('id'):
        if len(group) < 2:
            for measure_index, _ in group.iterrows():
                masked.append(measure_index)
    return masked

In [8]:
def compose_measure_name(measures):
    names = []
    for measure in measures:
        names.append(measure[0] + '|' + measure[1])
    return names

## Data Snooping Function

In [9]:
# k = 3.29 #critical values from Forstner
k = 4.1 #cricital value from Baarda

In [10]:
def data_snooping(network, sensors, parameters, k):
    """
    Parameters
    ----------
    network  :  DataFrame
                The control network as a dataframe generated by plio
    sensors  :  dict
                A dictionary that maps ISIS serial numbers to CSM sensors
    parameters  : list
                 The list of  CsmParameter to compute the partials W.R.T.
    k  :  float64
          Critical value used for rejection criteria; defaults to Forstner's 3.29 
          (or Baarda's 4.1??)
    
    Returns
    -------
      :  list
      Indices of the network DataFrame that were rejected during data snooping
    """
    net = network 
    net['mask'] = False

    rejected_indices = []
    awi = np.array([5, 5, 5, 5]) #initialize larger than k so you get into first iteration
    while (awi > k).any():
        print('number of measures remaining: ', len(net[~net['mask']]))

        # weight matrices
        coefficient_columns = compute_coefficient_columns(net[~net['mask']], sensors, parameters)
        num_parameters = max(col_range[1] for col_range in coefficient_columns.values())
        W_parameters = compute_parameter_weights(net[~net['mask']], sensors, parameters, coefficient_columns)
        num_observations = 2 * len(net[~net['mask']])
        W_observations = np.eye(num_observations)

        # bundle iteration (and set up)
        V = compute_residuals(net[~net['mask']], sensors)
        J = compute_jacobian(net[~net['mask']], sensors, parameters, coefficient_columns)
        sigma0 = compute_sigma(V, W_parameters, W_observations)
        N, dX = bundle_iteration(J, V, W_parameters, W_observations)

        # calculate test statistic
        wi = compute_normalized_residual(J, V, N, W_parameters, W_observations)
        awi = abs(wi)

        #find maximum
        imax = np.argmax(awi)
        print(f'max wi = {awi[imax]}') # display

        if awi[imax] <= k:
            print('Data Snooping Outlier Rejection Complete')
            break

        reject_index = floor(imax/2)
        print(f'max wi index = {imax}')
        print(f'max wi measure index = {reject_index}')
        reject = net.index[~net['mask']][reject_index]
        print(f'rejecting measure {net.loc[reject, ["id", "serialnumber"]].values}')
        net.loc[reject, ['mask']] = True
        rejected_indices.append(reject)
        
        not_enough_measures = check_network(net[~net['mask']])
        if (not_enough_measures):
            for measure_index in not_enough_measures:
                print(f'single measure point {net.loc[measure_index, "id"]}')
                print(f'rejecting measure {net.loc[measure_index, ["id", "serialnumber"]].values}')
                net.loc[measure_index, ['mask']] = True
        print('')
            
    return rejected_indices


In [11]:
results = {}

for net_file in glob.glob(os.path.join(network_dir, '*.net')):
    basename = os.path.splitext(net_file)[0]
    error_csv = basename + ".csv"
    try:
        error_df = pd.read_csv(error_csv, index_col=0)
    except:
        print(f'Skipping {net_file}')
        continue
    
    print(f'Starting {net_file}\n')
    network = io_controlnetwork.from_isis(net_file)
    compute_apriori_ground_points(network, sensors)
    rejected_indices = data_snooping(network, sensors, parameters, k)
    results[os.path.basename(net_file)] = (compose_measure_name(error_df[['id', 'serial']].values),
                                           compose_measure_name(network.loc[rejected_indices, ['id', 'serialnumber']].values))
    print('')

Starting /work/projects/control_network_metrics/registration_quality/measure_error_25px_2pts_1.5mean_1std.net

number of measures remaining:  70
max wi = 1.9183632568143518
Data Snooping Outlier Rejection Complete

Starting /work/projects/control_network_metrics/registration_quality/measure_error_5px_1pts_5mean_2std.net

number of measures remaining:  70
max wi = 2.4952268774901203
Data Snooping Outlier Rejection Complete

Starting /work/projects/control_network_metrics/registration_quality/measure_error_5px_1pts_1.5mean_1std.net

number of measures remaining:  70
max wi = 2.2070751819440995
Data Snooping Outlier Rejection Complete

Starting /work/projects/control_network_metrics/registration_quality/measure_error_50px_4pts_5mean_2std.net

number of measures remaining:  70
max wi = 3.29245741025329
Data Snooping Outlier Rejection Complete

Starting /work/projects/control_network_metrics/registration_quality/measure_error_50px_2pts_0mean_0.25std.net

number of measures remaining:  70
ma

In [12]:
for net_name, result in results.items():
    expected, actual = result
    missed = []
    found = []
    new = []
    for measure in expected:
        if measure in actual :
            found.append(measure)
        else:
            missed.append(measure)
    for measure in actual:
        if measure not in expected:
            new.append(measure) 
    print(net_name)
    print('Missed:', len(missed))
    print('Found: ', len(found))
    print('New:   ', len(new))
    print('')

measure_error_25px_2pts_1.5mean_1std.net
Missed: 2
Found:  0
New:    0

measure_error_5px_1pts_5mean_2std.net
Missed: 1
Found:  0
New:    0

measure_error_5px_1pts_1.5mean_1std.net
Missed: 1
Found:  0
New:    0

measure_error_50px_4pts_5mean_2std.net
Missed: 4
Found:  0
New:    0

measure_error_50px_2pts_0mean_0.25std.net
Missed: 2
Found:  0
New:    0

measure_error_25px_1pts_5mean_2std.net
Missed: 1
Found:  0
New:    0

measure_error_25px_4pts_0mean_0.25std.net
Missed: 4
Found:  0
New:    0

measure_error_50px_2pts_5mean_2std.net
Missed: 1
Found:  1
New:    0

measure_error_5px_1pts_0mean_0.25std.net
Missed: 1
Found:  0
New:    0

measure_error_50px_4pts_1.5mean_1std.net
Missed: 4
Found:  0
New:    0

measure_error_50px_2pts_1.5mean_1std.net
Missed: 2
Found:  0
New:    0

measure_error_5px_2pts_0mean_0.25std.net
Missed: 2
Found:  0
New:    0

measure_error_5px_4pts_1.5mean_1std.net
Missed: 4
Found:  0
New:    0

measure_error_25px_4pts_1.5mean_1std.net
Missed: 4
Found:  0
New:    0

m

In [13]:
for net_name, result in results.items():
    print(len(result[0]), len(result[1]), net_name)

2 0 measure_error_25px_2pts_1.5mean_1std.net
1 0 measure_error_5px_1pts_5mean_2std.net
1 0 measure_error_5px_1pts_1.5mean_1std.net
4 0 measure_error_50px_4pts_5mean_2std.net
2 0 measure_error_50px_2pts_0mean_0.25std.net
1 0 measure_error_25px_1pts_5mean_2std.net
4 0 measure_error_25px_4pts_0mean_0.25std.net
2 1 measure_error_50px_2pts_5mean_2std.net
1 0 measure_error_5px_1pts_0mean_0.25std.net
4 0 measure_error_50px_4pts_1.5mean_1std.net
2 0 measure_error_50px_2pts_1.5mean_1std.net
2 0 measure_error_5px_2pts_0mean_0.25std.net
4 0 measure_error_5px_4pts_1.5mean_1std.net
4 0 measure_error_25px_4pts_1.5mean_1std.net
1 1 measure_error_50px_1pts_0mean_0.25std.net
4 0 measure_error_25px_4pts_5mean_2std.net
2 0 measure_error_5px_2pts_5mean_2std.net
1 0 measure_error_25px_1pts_0mean_0.25std.net
4 0 measure_error_5px_4pts_5mean_2std.net
4 0 measure_error_50px_4pts_0mean_0.25std.net
2 0 measure_error_25px_2pts_0mean_0.25std.net
2 0 measure_error_25px_2pts_5mean_2std.net
1 2 measure_error_50px_1p